In [ ]:
from PIL import Image
import numpy as np
import scipy
import time
import math
import sys

In [ ]:
sys.path.append("build")
import evoapp

target_np = np.asarray(Image.open('lisa.png')).astype(np.float32) / 255.0

loss_cpu_state = evoapp.LossStateCPU(target_np)

NPrims = len(evoapp.DNA().primitives)
NVert = evoapp.Polygon.params() // 2

In [ ]:
def randarr():
    return np.random.rand(evoapp.DNA.params()).astype(np.float32)
def randna():
    return evoapp.DNA.fromarray(randarr())

In [ ]:
def loss(f):
    dna = evoapp.DNA.fromarray(f)
    return loss_cpu_state.loss(dna)
def loss_gpu(f):
    return loss(f)
    dna = evoapp.DNA.fromarray(f)
    return loss_gpu_state.loss(dna)

In [ ]:
def assert_consistent_eval():
    tmp_dna = randarr()
    l1 = loss_gpu(tmp_dna)
    l2 = loss_gpu(tmp_dna)
    assert l1 == l2
assert_consistent_eval()

In [ ]:
def benchmark_render(trials = 100):
    tmp_dna = randarr()
    start = time.time()
    for _ in range(trials):
        loss_gpu(tmp_dna)
    end = time.time()
    print(1.0 / ((end - start) / trials))
print(benchmark_render())

In [ ]:
dna = randna()

In [ ]:
best = randarr()

In [ ]:
best = np.load("build/best.npy")

In [ ]:
best_fun = loss(best)
improvements = []

print_interval = 100
i = 0
prev_improve_count = 0
prev_improve_amount = best_fun
start = time.time()
while True:
    data = np.copy(best)
    idx = np.random.randint(0,evoapp.DNA.params()) # medium
    # data[idx] = np.clip(data[idx] + (np.random.rand() - 0.5) * 0.25, 0, 1) # soft
    data[idx] = np.random.rand()
    result_fun = loss_gpu(data)
    if result_fun < best_fun:
        best = data
        best_fun = result_fun
        improvements.append(best_fun)
        prev_improve_count += 1
    
    i += 1
    if i % print_interval == 0:
        end = time.time()
        print("Iterations: {:.2f}/s | Improvements: {} ({:.2f}%) | Best: {:.2f}".format(print_interval / (end - start), prev_improve_count, float(100 * prev_improve_count) / print_interval, best_fun))
        prev_improve_count = 0
        prev_improve_amount = best_fun
        start = time.time()

In [ ]:
np.save("build/best.npy", best)

In [ ]:
loss(best)

In [ ]:
loss_gpu(best)

In [ ]:
Image.fromarray((evoapp.render(evoapp.DNA.fromarray(best)) * 255).astype(np.uint8))

In [ ]:
Image.fromarray((evoapp.render_gpu(evoapp.DNA.fromarray(best)) * 255).astype(np.uint8))

In [ ]:
# best = np.random.rand(evoapp.DNA.params()).astype(np.float32)
best_fun = loss_gpu(best)

i = 0
while True:
    try:
        data = randarr()
        result = scipy.optimize.minimize(loss_gpu,
                                         data,
                                         bounds=[(0, 1) for _ in range(evoapp.DNA.params())])
        if result.fun < best_fun:
            best = result.x
            best_fun = result.fun
            print('Best at iteration', i, 'is now', best_fun)
    except:
        pass
    i = i + 1

In [ ]:
result.fun

In [ ]:
Image.fromarray((target_np * 255).astype(np.uint8))

In [ ]:
result2 = scipy.optimize.minimize(loss_gpu, best, bounds=[(0, 1) for _ in range(evoapp.DNA.params())], jac='3-point', method='trust-constr')

In [ ]:
print(loss_gpu(best))
result2

In [ ]:
best = result2.x

In [ ]:
img = evoapp.render_gpu(evoapp.DNA.fromarray(best))
(np.min(img), np.max(img))

In [ ]:
np.max(target_np)

In [ ]:
# res = scipy.optimize.differential_evolution(loss_gpu, [(0, 1) for _ in range(evoapp.DNA.params())])
res

In [ ]:
results = []
while True:
    start = time.time()
    res = scipy.optimize.dual_annealing(loss_gpu, [(0, 1) for _ in range(evoapp.DNA.params())], maxfun=500000, no_local_search=True)
    results.append((res.fun, res.x, res))
    if res.fun < loss_gpu(best):
        best = res.x
        np.save("build/best_iter.npy", best)
    end = time.time()
    print(end - start, res.fun)


In [ ]:
Image.fromarray((evoapp.render_gpu(evoapp.DNA.fromarray(res.x)) * 255).astype(np.uint8))

In [ ]:
loss(best)

In [ ]:
best = res.x

In [ ]:
d = evoapp.DNA.fromarray(best)

In [ ]:
def serialize_dna(dna):
    dna_params = [NVert, NPrims]
    for p in dna.primitives:
        dna_params.extend(map(lambda i: int(i * 255), (p.r, p.g, p.b)))
        dna_params.append(0.5)
        verts = list(map(lambda i: round(i * 200), sum(map(lambda v: (v[1], v[0]), p.poly.transpose()), ())))
        dna_params.extend(verts)
    return ' '.join(map(str, dna_params))

In [ ]:
def deserialize_dna(str):
    opts = list(map(float, str.split(' ')))
    assert int(opts[0]) == NVert
    assert int(opts[1]) <= NPrims
    opts = opts[2:]

    d = evoapp.DNA()
    for i in range(NPrims):
        if len(opts) <= 2:
            break
        r, g, b, alpha, *verts = opts[0:(4 + 2 * NVert)]
        d.primitives[i].r = r / 255.0
        d.primitives[i].g = g / 255.0
        d.primitives[i].b = b / 255.0

        arr = np.asarray(verts).reshape((NVert, 2))[:, [1, 0]]
        d.primitives[i].poly = evoapp.Polygon(arr / 200.0)
        opts = opts[4 + 2 * NVert:]
    return d

In [ ]:
print(serialize_dna(d))
d2 = deserialize_dna(serialize_dna(d))
print(serialize_dna(d2))

In [ ]:
print(d2.primitives[2].poly.verts_x)
print(d2.primitives[0].poly.verts_y)

In [ ]:
# d2 = deserialize_dna('3 1 0 64 128 0.5 0 0 199 199 199 0')
d2 = deserialize_dna('3 50 141 167 110 0.7692422436853981 200 200 48 198 200 0 136 159 104 0.78125 0 0 62 182 200 0 45 29 18 0.6085522079420671 69 16 183 125 139 14 56 44 28 0.8445853674912092 18 200 156 31 199 200 60 70 36 0.83984375 106 200 0 1 0 200 65 49 43 0.1130273745411643 0 0 111 0 40 169 72 79 57 0.77734375 98 8 200 200 200 90 65 31 38 0.1484375 199 83 101 62 161 48 255 182 96 0.6796875 25 65 99 200 132 59 187 196 140 0.9244396412892593 114 54 0 102 1 14 249 196 101 0.83203125 145 200 130 156 62 200 119 87 46 0.98046875 35 80 122 137 132 78 250 201 115 0.87109375 52 99 107 127 129 89 98 72 39 0.421875 78 115 105 187 176 192 252 221 139 0.7109375 80 100 107 37 58 38 16 11 21 0.27734375 118 178 7 107 49 163 255 189 91 0.2578125 68 152 123 148 135 82 253 230 154 0.25080006806528976 59 125 118 46 36 34 64 66 43 0.3510387900285945 158 93 174 169 48 199 86 92 64 0.98828125 0 159 56 38 55 90 112 90 37 0.19921875 114 40 146 59 99 70 61 56 37 0.55859375 189 168 31 172 96 152 222 242 179 0.359375 37 114 43 40 0 75 132 122 121 0.46875 64 3 135 26 172 39 104 53 2 0.14453125 68 20 138 95 134 4 91 119 91 0.08984375 165 177 81 70 196 106 240 160 62 0.23046875 83 125 126 53 104 37 125 150 122 0.57421875 42 150 28 199 0 166 126 148 92 0.296875 16 49 0 0 80 0 229 172 45 0.10546875 103 35 135 71 32 190 255 139 175 0.03515625 117 27 195 159 81 131 64 66 56 0.33203125 120 94 155 30 116 21 82 81 52 0.73828125 57 58 44 187 18 120 60 62 59 0.109375 30 33 18 72 119 22 79 42 52 0.28515625 54 103 20 199 81 198 83 36 4 0.26953125 23 95 81 8 38 89 78 66 30 0.703125 57 35 100 9 34 77 89 56 46 0.17578125 85 104 77 146 129 130 254 178 88 0.2734375 67 37 46 61 77 157 71 13 7 0.1640625 23 146 64 20 55 122 143 242 149 0.5859375 81 127 45 58 64 94 98 7 0 0.09375 88 112 65 68 116 92 244 237 42 0.5546875 133 101 19 171 167 80 76 74 41 0.984375 39 176 0 199 27 200 115 82 48 0.5390625 175 199 139 200 132 173 57 46 49 0.2578125 184 199 117 120 166 53 160 177 134 0.73828125 0 54 62 31 0 2 156 147 134 0.00390625 97 165 149 55 60 101 112 136 96 0.63671875 166 61 200 68 200 124 245 251 121 0.08573622919711199 54 58 118 146 117 103')
print(serialize_dna(d2))

In [ ]:
Image.fromarray((evoapp.render(d2) * 255).astype(np.uint8))

In [ ]:
Image.fromarray((evoapp.render(d2) * 255).astype(np.uint8))

In [ ]:
best_fun = loss_gpu(best)
print(best_fun)
for i in range(evoapp.DNA.params()):
    data = np.copy(best)
    loss_prev = best_fun
    steps_remain = 100
    while steps_remain > 0:
        prev = data[i]
        data[i] = np.clip(data[i] + (np.random.rand() - 0.5) * 0.1, 0, 1)
        loss_curr = loss_gpu(data)
        if loss_curr < loss_prev:
            loss_prev = loss_curr
            steps_remain += 10
        else:
            data[i] = prev
            steps_remain -= 1
    if loss_prev < best_fun:
        best_fun = loss_prev
        best = data
print(loss_gpu(best))

In [ ]:
evoapp.Polygon(np.asarray([[0, 0], [0, 0], [0, 0]])).test(0.5, 0.5)